In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
import wandb
from tabulate import tabulate

## Uploading files to HF

In [2]:
import os
from huggingface_hub import login
token = os.getenv("HUGGINGFACE_TOKEN")
login(token=token)

In [3]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# import wandb

# # Step 1: Get the model from W&B artifact
# run = wandb.init()
# artifact = run.use_artifact(
#     "medoxz543-zewail-city-of-science-and-technology/bertweet-lora-bayes-v2/final_model:v5",
#     type="model"
# )
# artifact_dir = artifact.download()

# # Step 2: Load model and tokenizer
# model = AutoModelForSequenceClassification.from_pretrained(artifact_dir)
# tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

# # Step 3: Save them in .bin format (NOT safetensors)
# model.save_pretrained("./hf_model_bin", safe_serialization=False)
# tokenizer.save_pretrained("./hf_model_bin")

In [4]:
# from huggingface_hub import upload_folder

# upload_folder(
#     repo_id="medoxz543/hate-speech",
#     folder_path="./hf_model_bin",
#     commit_message="✅ Uploading model in .bin format for Inference API"
# )

## Testing HF endpoint

In [4]:
import requests
import time

API_URL = "https://medoxz543-hate-endpoint.hf.space/check-text"
HEADERS = {"Content-Type": "application/json"}

sample_payload = {
    "texts": [
        "Jews are killers. We should kill them all",
        "Have a nice day!",
        "Go back to where you came from.",
        "Women are nasty creatures."
    ]
}

start_time = time.time()

try:
    response = requests.post(API_URL, headers=HEADERS, json=sample_payload)
    response.raise_for_status()
    elapsed = time.time() - start_time

    print("✅ Prediction Response:", response.json())
    print(f"⏱️ Inference Time: {elapsed:.3f} seconds")

except requests.exceptions.RequestException as e:
    elapsed = time.time() - start_time
    print("❌ Failed to call API:", e)
    print(f"⏱️ Elapsed Time: {elapsed:.3f} seconds")
    if response is not None:
        print("Response text:", response.text)

✅ Prediction Response: {'timestamp': '2025-05-27T14:34:30', 'results': [{'blur': True, 'score': 0.9555}, {'blur': False, 'score': 0.0067}, {'blur': False, 'score': 0.23}, {'blur': True, 'score': 0.9063}]}
⏱️ Inference Time: 4.291 seconds


In [6]:
import requests
import time

API_URL = "https://medoxz543-hate-endpoint.hf.space/log-results"
HEADERS = {"Content-Type": "application/json"}

sample_payload = {
    "texts": [
        "You're such a loser.",
        "Have a nice day!",
        "Go back to where you came from.",
        "Your work is brilliant, truly impressive!"
    ]
}

start_time = time.time()

try:
    response = requests.post(API_URL, headers=HEADERS, json=sample_payload)
    response.raise_for_status()
    elapsed = time.time() - start_time

    print("✅ Log Response:", response.json())
    print(f"⏱️ Logged Inference Time: {elapsed:.3f} seconds")

except requests.exceptions.RequestException as e:
    elapsed = time.time() - start_time
    print("❌ Failed to log results:", e)
    print(f"⏱️ Elapsed Time: {elapsed:.3f} seconds")
    if response is not None:
        print("Response text:", response.text)


✅ Log Response: {'logged': True}
⏱️ Logged Inference Time: 1.615 seconds


## The sample comparison method

In [1]:
from model_eval import CASE_STUDY

In [2]:
cs = CASE_STUDY()
# cs.describe_df()
cs.populate_df()

Evaluating model: Hate-speech-CNERG/dehatebert-mono-english
Evaluating model: ctoraman/hate-speech-bert
Evaluating model: facebook/roberta-hate-speech-dynabench-r4-target
Evaluating model: cardiffnlp/twitter-roberta-base-offensive
Evaluating model: cardiffnlp/twitter-roberta-base-hate-latest
Evaluating model: medoxz543/hate-speech


In [3]:
cs.pivot_df

,ableism,anti_lgbtq,dehumanization,islamophobia,not_hateful,other_religion_hate,political_hate,racism,sexism,socioeconomic_hate,xenophobia,hate_avg
Hate-speech-CNERG/dehatebert-mono-english,0.00,0.05,0.35,0.20,0.95,0.05,0.20,0.15,0.05,0.30,0.30,0.16
ctoraman/hate-speech-bert,0.15,0.15,0.40,0.05,1.00,0.35,0.45,0.20,0.00,0.35,0.05,0.21
facebook/roberta-hate-speech-dynabench-r4-target,0.30,0.55,0.00,0.90,1.00,0.90,0.10,0.50,0.70,0.25,0.50,0.47
cardiffnlp/twitter-roberta-base-offensive,0.40,0.70,1.00,0.55,0.95,0.75,0.80,0.70,0.35,0.75,0.30,0.63
cardiffnlp/twitter-roberta-base-hate-latest,0.05,0.10,0.30,0.40,1.00,0.20,0.30,0.20,0.60,0.10,0.40,0.27
medoxz543/hate-speech,0.25,0.35,0.40,0.65,1.00,0.65,0.45,0.60,0.65,0.25,0.40,0.47


In [4]:
cs.show_nonhate_and_hate()

In [5]:
cs.show_heatmap()

In [7]:
from transformers import AutoConfig
import pandas as pd

# Model names
model_names = [
    "cardiffnlp/twitter-roberta-base-offensive",
    "medoxz543/hate-speech"
]

# Load configurations only (faster than full model loading)
model_configs = {name: AutoConfig.from_pretrained(name) for name in model_names}

# Extract relevant architecture information
model_arch_info = {
    name: {
        "architectures": config.architectures,
        "model_type": config.model_type,
        "hidden_size": config.hidden_size,
        "num_attention_heads": config.num_attention_heads,
        "num_hidden_layers": config.num_hidden_layers
    }
    for name, config in model_configs.items()
}

# Convert to DataFrame for easy comparison
arch_df = pd.DataFrame(model_arch_info).T
arch_df


,architectures,model_type,hidden_size,num_attention_heads,num_hidden_layers
cardiffnlp/twitter-roberta-base-offensive,[RobertaForSequenceClassification],roberta,768,12,12
medoxz543/hate-speech,[RobertaForSequenceClassification],roberta,768,12,12


In [8]:
from transformers import AutoTokenizer

tok1 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-offensive")
tok2 = AutoTokenizer.from_pretrained("medoxz543/hate-speech")

print(tok1.vocab_size, tok2.vocab_size)
print(tok1.model_max_length, tok2.model_max_length)


50265 64000
1000000000000000019884624838656 1000000000000000019884624838656


In [11]:
from transformers import AutoModel, AutoConfig
import torch

# Define the model names
model_roberta = "cardiffnlp/twitter-roberta-base-offensive"
model_bertweet = "medoxz543/hate-speech"

# Load models and configs
model_r = AutoModel.from_pretrained(model_roberta)
model_b = AutoModel.from_pretrained(model_bertweet)

config_r = model_r.config
config_b = model_b.config

# Basic details
print("\n📌 Base Classes")
print("RoBERTa base model:", model_r.base_model.__class__)
print("BERTweet base model:", model_b.base_model.__class__)

print("\n📌 Number of Parameters")
print("RoBERTa total parameters:", sum(p.numel() for p in model_r.parameters()) / 1e6, "M")
print("BERTweet total parameters:", sum(p.numel() for p in model_b.parameters()) / 1e6, "M")

print("\n📌 Core Config Comparison")
for key in ["hidden_size", "num_attention_heads", "num_hidden_layers", "intermediate_size",
            "vocab_size", "max_position_embeddings", "type_vocab_size", "layer_norm_eps"]:
    print(f"{key:>30}: RoBERTa={getattr(config_r, key)} | BERTweet={getattr(config_b, key)}")

print("\n📌 Dropout and Activation")
print(f"hidden_dropout_prob    : RoBERTa={config_r.hidden_dropout_prob} | BERTweet={config_b.hidden_dropout_prob}")
print(f"attention_dropout_prob : RoBERTa={config_r.attention_probs_dropout_prob} | BERTweet={config_b.attention_probs_dropout_prob}")
print(f"hidden_act             : RoBERTa={config_r.hidden_act} | BERTweet={config_b.hidden_act}")

print("\n📌 Positional Embeddings Type (RoBERTa has sinusoidal)")
print("RoBERTa positional embeddings:", hasattr(model_r.embeddings, 'position_embeddings'))
print("BERTweet positional embeddings:", hasattr(model_b.embeddings, 'position_embeddings'))


Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-offensive and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at medoxz543/hate-speech and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



📌 Base Classes
RoBERTa base model: <class 'transformers.models.roberta.modeling_roberta.RobertaModel'>
BERTweet base model: <class 'transformers.models.roberta.modeling_roberta.RobertaModel'>

📌 Number of Parameters
RoBERTa total parameters: 124.645632 M
BERTweet total parameters: 134.899968 M

📌 Core Config Comparison
                   hidden_size: RoBERTa=768 | BERTweet=768
           num_attention_heads: RoBERTa=12 | BERTweet=12
             num_hidden_layers: RoBERTa=12 | BERTweet=12
             intermediate_size: RoBERTa=3072 | BERTweet=3072
                    vocab_size: RoBERTa=50265 | BERTweet=64001
       max_position_embeddings: RoBERTa=514 | BERTweet=130
               type_vocab_size: RoBERTa=1 | BERTweet=1
                layer_norm_eps: RoBERTa=1e-05 | BERTweet=1e-05

📌 Dropout and Activation
hidden_dropout_prob    : RoBERTa=0.1 | BERTweet=0.1
attention_dropout_prob : RoBERTa=0.1 | BERTweet=0.1
hidden_act             : RoBERTa=gelu | BERTweet=gelu

📌 Positional Embedd

In [10]:
import pandas as pd

df = pd.read_csv("data/sexism/sexism_data.csv")
df

,id,dataset,text,toxicity,sexist,of_id
0,0,other,MENTION3481 i didn't even know random was an o...,0.118180,False,-1
1,1,other,Bottom two should've gone! #mkr,0.251850,False,-1
2,2,callme,MENTION3111 MENTION3424 ladyboner deserves so ...,0.113331,False,-1
3,3,other,She shall now be known as Sourpuss #MKR #KatAn...,0.531153,False,-1
4,4,other,Tarah W threw a bunch of women under the bus s...,0.118718,False,-1
...,...,...,...,...,...,...
13626,13630,callme,this reminds me of the MENTION3079 situation; ...,0.147044,False,-1
13627,13631,other,#mkr I love Annie and loyld there like a real ...,0.213106,False,-1
13628,13632,other,No u. http://t.co/zOr0eWahSS,0.324702,False,-1
13629,13633,other,#mkr the way kat looks at Annie is like she's ...,0.563036,False,-1
